In [ ]:
import os
import csv
from scapy.all import rdpcap, TCP

def calculate_rtt(packets):
    syn_times = {}
    rtt_list = []

    for pkt in packets:
        if pkt.haslayer(TCP):
            if pkt[TCP].flags == "S":  # SYN
                syn_times[pkt[TCP].seq] = pkt.time
            elif pkt[TCP].flags == "SA":  # SYN-ACK
                if pkt[TCP].ack - 1 in syn_times:
                    rtt = pkt.time - syn_times[pkt[TCP].ack - 1]
                    rtt_list.append(rtt)
    return rtt_list

def calculate_bandwidth(packets, duration):
    total_bytes = sum(len(pkt) for pkt in packets)
    duration = float(duration)
    bandwidth = (total_bytes * 8) / duration
    return bandwidth / 1e6

def calculate_jitter(rtt_list):
    return [abs(rtt_list[i] - rtt_list[i - 1]) for i in range(1, len(rtt_list))]

def detect_packet_loss(packets):
    seen_seq = set()
    lost_packets = 0
    for pkt in packets:
        if pkt.haslayer(TCP):
            seq_num = pkt[TCP].seq
            if seq_num in seen_seq:
                lost_packets += 1
            seen_seq.add(seq_num)
    return lost_packets

# Base path where the datasets are stored
base_path = "dataset"
connections = ["5G connection", "Wired connection"]
games = ["Fortnite", "Forza Horizon 5", "Mortal Kombat 11"]
modes = ["1 Player", "2 Players"]

results = [["Connection", "Game", "Mode", "Avg RTT (s)", "Bandwidth (Mbps)", "Avg Jitter (s)", "Packet Loss (packets)"]]

for conn in connections:
    for game in games:
        for mode in modes:
            pcap_path = os.path.join(base_path, conn, game, mode)
            if not os.path.isdir(pcap_path):
                continue

            # Find all .pcap files in the directory
            pcap_files = [f for f in os.listdir(pcap_path) if f.endswith(".pcap")]
            if not pcap_files:
                continue

            # Initialize accumulators for the metrics
            total_rtt = []
            total_jitter = []
            total_bytes = 0
            total_duration = 0
            total_loss = 0
            
            for pcap_file in pcap_files:
                full_pcap_path = os.path.join(pcap_path, pcap_file)
                try:
                    packets = rdpcap(full_pcap_path)

                    # Calculate metrics for each pcap file
                    rtt_list = calculate_rtt(packets)
                    jitter_list = calculate_jitter(rtt_list)
                    duration = packets[-1].time - packets[0].time
                    bw = calculate_bandwidth(packets, duration)
                    loss = detect_packet_loss(packets)

                    # Aggregate metrics
                    if rtt_list:
                        total_rtt.extend(rtt_list)
                    if jitter_list:
                        total_jitter.extend(jitter_list)
                    total_bytes += sum(len(pkt) for pkt in packets)
                    total_duration += duration
                    total_loss += loss

                except Exception as e:
                    print(f"Failed to read {full_pcap_path}: {e}")
                    continue

            # Calculate averages for the aggregated results
            avg_rtt = sum(total_rtt) / len(total_rtt) if total_rtt else 0
            avg_jitter = sum(total_jitter) / len(total_jitter) if total_jitter else 0
            avg_bw = (total_bytes * 8) / float(total_duration) / 1e6 if total_duration > 0 else 0

            # Append results for this game and mode
            results.append([
                conn.replace(" connection", ""),
                game,
                mode,
                f"{avg_rtt:.4f}",
                f"{avg_bw:.2f}",
                f"{avg_jitter:.4f}",
                total_loss
            ])

# Save to CSV
with open("network_metrics_summary.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(results)

print("All results saved to network_metrics_summary.csv")


All results saved to network_metrics_summary.csv
